In [ ]:
pip install transformers torch datasets scikit-learn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from datasets import Dataset

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Project/balanced_train.csv')
data = data.drop(columns=['id', 'is_toxic'])
data

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"""\nNo, it doesn't do fill-in. In handles """"pat...",0,0,0,0,0,0
1,"Well, if The Tartan, a sometime student public...",0,0,0,0,0,0
2,"hi, friends, should we talk about the line mar...",0,0,0,0,0,0
3,"""\nYes, my cat was subject to attempt to test ...",0,0,0,0,0,0
4,2012 (UTC)\nTry The Books of the Bible. 16:0...,0,0,0,0,0,0
...,...,...,...,...,...,...,...
47364,"""\n\n The LGBT Barnstar awarded because your ...",0,0,0,0,0,1
47365,"""\n\nHow genius! RickK would love your style ...",0,0,0,0,0,1
47366,"LWC, there is more to it than you think. You ...",0,0,0,0,0,1
47367,"Basically, homosexuality lowers productivity, ...",0,0,0,0,0,1


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from pylab import rcParams

In [ ]:
# Convert the HuggingFace Dataset to a Pandas DataFrame
data_df = pd.DataFrame(data)

# Visualizing the class distribution of the 'label' column
column_labels = data_df.columns.tolist()[2:]
label_counts = data_df[column_labels].sum().sort_values()

# Create a black background for the plot
plt.figure(figsize=(7, 5))

# Create a horizontal bar plot using Seaborn
ax = sns.barplot(x=label_counts.values,
y=label_counts.index, palette='viridis')

# Add labels and title to the plot
plt.xlabel('Number of Occurrences')
plt.ylabel('Labels')
plt.title('Distribution of Label Occurrences')

# Show the plot
plt.show()

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['comment_text'], padding='max_length', truncation=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
data = Dataset.from_pandas(data)
data = data.map(tokenize_function, batched=True)
data = data.rename_column('comment_text', 'text')

Map:   0%|          | 0/47369 [00:00<?, ? examples/s]

In [ ]:
# Train/test split
train_dataset, test_dataset = data.train_test_split(test_size=0.2).values()

In [ ]:
# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

In [ ]:
# Load model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=1,
    weight_decay=0.01,
)

# Custom Trainer class to handle multi-label classification
class MultiLabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(outputs.logits, labels)
        return (loss, outputs) if return_outputs else loss

# Prepare labels for training
def prepare_labels(batch):
    labels = torch.stack([batch['toxic'], batch['severe_toxic'], batch['obscene'], batch['threat'], batch['insult'], batch['identity_hate']], dim=1).float()
    batch["labels"] = labels
    return batch

train_dataset = train_dataset.map(prepare_labels, batched=True)
test_dataset = test_dataset.map(prepare_labels, batched=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/37895 [00:00<?, ? examples/s]

Map:   0%|          | 0/9474 [00:00<?, ? examples/s]

In [ ]:
# Trainer
trainer = MultiLabelTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# Train model
#trainer.train()

In [ ]:
# Evaluate model
eval_result = trainer.evaluate()
print(f"Evaluation result: {eval_result}")

Evaluation result: {'eval_loss': 0.815601646900177, 'eval_runtime': 277.1909, 'eval_samples_per_second': 34.179, 'eval_steps_per_second': 4.275}


In [ ]:
#Saving Model
#trainer.save_model('/content/drive/MyDrive/Project/saved_model')

In [ ]:
#Loading Model
model = BertForSequenceClassification.from_pretrained('/content/drive/MyDrive/Project/saved_model')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

predictions = trainer.predict(test_dataset)
y_pred = torch.sigmoid(torch.tensor(predictions.predictions)).numpy() > 0.5

y_true = torch.stack([test_dataset['toxic'], test_dataset['severe_toxic'], test_dataset['obscene'],
                      test_dataset['threat'], test_dataset['insult'], test_dataset['identity_hate']], dim=1).numpy()

accuracy = accuracy_score(y_true.flatten(), y_pred.flatten())
precision = precision_score(y_true.flatten(), y_pred.flatten())
recall = recall_score(y_true.flatten(), y_pred.flatten())
f1 = f1_score(y_true.flatten(), y_pred.flatten())

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


KeyboardInterrupt: 

In [ ]:
def classify_comment(comment):
    inputs = tokenizer(comment, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
    # Move inputs to the same device as the model
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    outputs = model(**inputs)
    # Move the logits to the CPU before converting to a NumPy array
    predictions = torch.sigmoid(outputs.logits).cpu().detach().numpy()
    return predictions

new_comment = "i will kill you."
predictions = classify_comment(new_comment)
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
predicted_labels = {label: prediction for label, prediction in zip(labels, predictions[0])}
print(f"Comment: '{new_comment}' has predictions: {predicted_labels}")

Comment: 'i will kill you.' has predictions: {'toxic': 0.8180087, 'severe_toxic': 0.1944666, 'obscene': 0.16288623, 'threat': 0.81650305, 'insult': 0.39069295, 'identity_hate': 0.16202207}


In [ ]:
!pip install gradio
import gradio as gr

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 857.8/857.8 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.1/318.1 kB 33.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 10.

In [ ]:
def classify_comment(comment):
  inputs = tokenizer(comment, return_tensors='pt', padding='max_length', truncation=True, max_length=128)
  inputs = {k: v.to(model.device) for k, v in inputs.items()}
  outputs = model(**inputs)
  predictions = torch.sigmoid(outputs.logits).cpu().detach().numpy()
  labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
  predicted_labels = {label: prediction for label, prediction in zip(labels, predictions[0])}
  return predicted_labels

demo = gr.Interface(
    fn=classify_comment,
    inputs=gr.Textbox(lines=5, label="Enter Comment"),
    outputs=gr.Label(label="Toxicity Analysis"),
)

demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://7a4664a659beb4393a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
